In [1]:
import mysql.connector
import scrapy
from scrapy.crawler import CrawlerProcess
from multiprocessing import Process, Queue
from twisted.internet import reactor
import logging
import re
import time
import html2text
from datetime import datetime
from dateutil import parser

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

date_parser = '%Y'

In [ ]:
conn = mysql.connector.connect(host='localhost', database='discogs3', user='root', password='root')
# if conn.is_connected():
cursor = conn.cursor()
#     cursor.execute('SELECT * FROM artist')
#     print(cursor.fetchone())
    
# cursor.close()
# conn.close()

def add_artist(*args):
#     print("DEBUG\t", args[1], args[2])
    if type(args[0]) == tuple:
        try:
            print(args[0])
            args[0] = None
            args[0] = 0
        except:
            args[0] = 0
        
    cursor.execute("""SELECT id FROM artist WHERE id="{0}" """.format(args[0]))
    if cursor.fetchone() is None:
        try:
            cursor.execute("""INSERT INTO artist VALUES (%s, %s, %s); """, (args[0], args[1], args[2][:500] if len(args[2]) >= 500 else args[2]))
            conn.commit()
        except Exception as err:
            print('artist', arg[0], err)
            conn.rollback()
        
def add_album(*args, released=True):
    try:
#         print(args[0], args[1], args[2])
        if released:
            try:
                qwer = parser.parse(args[3])
            except Exception as err:
#                 print('jeasdasd', err)
                qwer = None
            
            cursor.execute("""INSERT INTO album (artist_id, album_name, versions, released, country, rating, format) VALUES (%s, %s, %s, %s, %s, %s, %s)""", (args[0], args[1], args[2], qwer, args[4], args[5], args[6],)) # id, artist_id, name, ver, rel, count, rating
        else:
            cursor.execute("""INSERT INTO album (artist_id, album_name, versions, country, rating, format) VALUES (%s, %s, %s, %s, %s, %s)""", (args[0], args[1], args[2], args[3], args[4], args[5],))

        conn.commit()
        return cursor.lastrowid
    except Exception as err:
        print ('album', args[0], args[1], err)
        conn.rollback()
        
def add_style(*args):
    for s in args[0]:
        if s:
            try:
                cursor.execute("""INSERT INTO album_style (album_id, style) VALUES (%s, %s)""", (args[1], s,))
                conn.commit()
            except Exception as err:
                print ('style', args[1], err)
                conn.rollback()

def add_genre(*args):
    for g in args[0]:
        if g:
            try:
                cursor.execute("""INSERT INTO album_genre (album_id, genre) VALUES (%s, %s)""", (args[1], g,))
                conn.commit()
            except Exception as err:
                print ('genre', args[1], err)
                conn.rollback()

def add_tracklist(*args):
    for t in args[0]:
        try:
            sec = 0
            if t[1]:
                sec = int((t[1][0] - datetime.strptime("0:0", "%M:%S")).total_seconds())
            
            if len(t[0]) > 300:
                t[0] = t[0][:300]
                
            cursor.execute("""INSERT INTO track_list (album_id, track_name, duration) VALUES (%s, %s, %s)""", (args[1], t[0], sec,))
            conn.commit()
        except Exception as err:
            print('tracks', args[1], err)
            conn.rollback()

def add_album_vocals(*args):
    for v in args[0]:
        try:
#             print('vocals, ', v)
            cursor.execute("""INSERT INTO album_vocals (album_id, artist_id) VALUES (%s, %s)""", (args[1], v,))
            conn.commit()
        except Exception as err:
            print('alb_vocals', args[1], err)
            conn.rollback()
            
def add_album_other(*args):
    for v in args[0]:
        try:
#             print('vocals, ', v)
            cursor.execute("""INSERT INTO album_other (album_id, artist_id) VALUES (%s, %s)""", (args[1], v,))
            conn.commit()
        except Exception as err:
            print('alb_other', args[1], err)
            conn.rollback()
            
def add_track_music(arg):
    for i in arg:
        try:
            cursor.execute("""INSERT INTO track_music (artist_id) VALUES (%s)""", (i,))
            conn.commit()
        except Exception as err:
            print('music_t', err)
            conn.rollback()
            
def add_track_lyrics(arg):
    for i in arg:
        try:
            cursor.execute("""INSERT INTO track_lyrics (artist_id) VALUES (%s)""", (i,))
            conn.commit()
        except Exception as err:
            print ('lyrics_t', err)
            conn.rollback()
            
def add_track_arrangement(arg):
    for i in arg:
        try:
            cursor.execute("""INSERT INTO track_arrangement (artist_id) VALUES (%s)""", (i,))
            conn.commit()
        except Exception as err:
            print ('arr_t', err)
            conn.rollback()

# Crawler

In [ ]:
class DiscogsSpider(scrapy.Spider):
    name = "pauk"
    allowed_domains = ['discogs.com']
    start_url = "https://www.discogs.com/search/?limit=250&country_exact={}&page=1"
    
    def __init__(self, *args, **kwargs):
        self.country = kwargs.get('country')
        super(DiscogsSpider, self).__init__(*args, **kwargs)
        
    def start_requests(self):
        yield scrapy.Request(url=self.start_url.format(self.country), callback=self.parse)
        
    def parse(self, response):
        for page in response.xpath('//a[@class="search_result_title"]/@href'):
            yield response.follow(page, callback=self.parse_data)
            
        yield response.follow(response.xpath('//a[@class="pagination_next"]')[0], callback=self.parse)
        
    def parse_artist(self, response):
        identificator = response.request.url.split('/')[-1].split('-')[0]
        name = response.selector.xpath("//div[@class='profile']/h1/text()").getall()[0]
        sites = response.selector.xpath("//div[@class='profile']/div[@class='content'][2]/a/@href").getall()
        
        ### check if artist exists and insert if not
        add_artist(identificator, name, ",".join(sites))
        
        yield None
        
    def parse_data(self, response):
        def parse_rating():
            try:
                rating = response.selector.xpath("//span[@class='rating_value']/text()").getall()[0]
                rating = float(rating)
            except ValueError:
                rating = 0.0
            return rating
        
        def parse_credits():
            def role(pattern, roles, selector):
                if roles == []:
                    return []

                regex = re.compile(pattern, re.IGNORECASE)
                if regex.search(roles):
                    try:
                        person = selector.xpath("./a[@class='rollover_link']/@href").get().split('/')[2].split('-')[0]
                        return person
                    except:
                        return []

                return []
            arrangement = []
            writters = []
            vocals = []
            for selector in response.selector.xpath("//ul[@class='list_no_style']/li"):
                roles = selector.xpath("./span[@class='role']/text()").extract()
                roles = roles[0] if roles != [] else ""

                res = role('vocals', roles, selector)
                vocals += list(res)

                res = role('written', roles, selector)
                writters += list(res)

                res = role('arranged', roles, selector)
                arrangement += list(res)

            return {'vocals': vocals, 'writting': writters, 'arrangement': arrangement}
        
        def versions():
            ver_num = len(response.selector.xpath('//table[@id="versions"]/tr'))
            return 1 if ver_num == 0 else ver_num - 1
        
        def album_name():
            try:
                ret = response.xpath('//div[@class="profile"]/h1/span[2]/text()').getall()[0].strip()
            except:
                ret = ""
            
            return ret
        
        def artist_id():
            try:
                ret = response.xpath('//div[@class="profile"]/h1/span[1]/span/a/@href').get().split('/')[2].split('-')[0]
            except:
                ret = ""
#             print('debug', response, response.xpath('//div[@class="profile"]/h1/span[1]').getall())
            return ret
        
        def album_profile():
            header_list = response.selector.xpath("//div[@class='profile']/div[@class='head']/text()").extract()
            content_selectors = response.selector.xpath("//div[@class='profile']/div[@class='content']")

            if len(header_list) != len(content_selectors):
                if self._err_recorder is not None:
                    self._err_recorder.report_possible_error(response.url, "Profile data")
                return {}

            converter = html2text.HTML2Text()
            converter.ignore_links = True

            data = {}
            for i in range (0, len(content_selectors)):
                contend = converter.handle(content_selectors[i].extract())
                data[header_list[i].replace(':', '')] = str.strip(contend)
            
            data['album_id'] = response.request.url.split('/')[-1]
            return data
        
        def tracks():
            tracklist_selectors = response.selector.xpath("//tr[@class=' tracklist_track track']")
            tracklist_selectors.append(response.selector.xpath("//tr[@class='first tracklist_track track']"))

            data = []
            for selector in  tracklist_selectors:
                title = selector.xpath("./td[@class='track tracklist_track_title '] |  ./td[@class='track tracklist_track_title mini_playlist_track_has_artist']").xpath("./a/span/text() | ./span/text()").extract()

                if title == []:
                    continue
                title = title[0].strip()

                duration = selector.xpath("./td[@class='tracklist_track_duration']/span/text()").extract()
                if duration != []:
                    try:
                        duration = [datetime.strptime(duration[0].strip(), '%M:%S')]
                    except:
                        try:
                            duration = [datetime.strptime(duration[0].strip(), '%H:%M:%S')]
                        except:
                            duration = [0]
                else:
                    duration = None

                data.append([title, duration])

            return data

        def track_credits():
            def role(pattern, roles, selector):
                if roles == []:
                    return []

                regex = re.compile(pattern, re.IGNORECASE)
                if regex.search(roles):
                    tmp = selector.xpath("./a/@href").getall()
                    person = []
                    links = []
                    for p in tmp:
                        try:
                            links.append(p)
                            person.append(int(p.split('/')[2].split('-')[0]))
                        except:
                            person.append(0)
                            
                    return person, links

                return [], []
            
            selector = response.selector.xpath("//span[@class='tracklist_extra_artist_span']")
            arrangement = []
            lyrics = []
            music = []
            links = []
            
            for sel in selector:
                roles = selector.xpath("./text()").get()
                res = role('lyrics', roles, sel)
                lyrics += res[0]
                links += list(set(res[1]))
                
                res = role('music', roles, sel)
                music += res[0]
                links += list(set(res[1]))
                
                res = role('arranged', roles, sel)
                arrangement += res[0]
                links += res[1]
                
#             print(music, lyrics, arrangement, list(set(links)))
            return {'music': music, 'lyrics': lyrics, 'arrangement': arrangement, 'links': links}
        
        for artist in response.xpath('//div[@class="profile"]/h1/span/span/a/@href'):
#             print('ko je ovde lud', artist)
            yield response.follow(artist, callback=self.parse_artist)
  
        track_credits = track_credits()
        for x in track_credits['links']:
            yield response.follow(x, callback=self.parse_artist)
        
        data = {"rating": parse_rating(), "album_credits": parse_credits(), "versions": versions(), "album_name": album_name(), 
               "artist_id": artist_id(), "album_profile": album_profile(), "tracks": tracks(), "track_credits": track_credits}
#         print(data['album_profile'])
    
        if type(data['artist_id']) == tuple:
            data['artist_id'] = data['artist_id'][0]
            data['artist_id'] = 0
        if data['artist_id'] == 'Various?anv=' or data['artist_id'] == 'Unknown+Artist?anv=':
            data['artist_id'] = '0'
    
        if 'Released' in data['album_profile']:
            album_id = add_album(data['artist_id'], data['album_name'], data['versions'], data['album_profile']['Released'], data['album_profile'].get('Country', ""), data['rating'], data['album_profile'].get('Format', None))
        else:
            album_id = add_album(data['artist_id'], data['album_name'], data['versions'], data['album_profile'].get('Country', ""), data['rating'], data['album_profile'].get('Format', None), released=False)
        
        data['album_profile']['album_id'] = album_id
        add_style(data['album_profile']['Style'].split(','), data['album_profile']['album_id'])
        add_genre(data['album_profile']['Genre'].split(','), data['album_profile']['album_id'])
        
        add_tracklist(data['tracks'], data['album_profile']['album_id'])
        
        other = list(set(data['album_credits']['arrangement'] + data['album_credits']['writting']))
        add_album_vocals(list(set(data['album_credits']['vocals'])), data['album_profile']['album_id'])
        add_album_other(other, data['album_profile']['album_id'])
#         add_album_other(data['album_credits']['writting'], data['album_profile']['album_id'])
        
        add_track_music(track_credits['music'])
        add_track_lyrics(track_credits['lyrics'])
        add_track_arrangement(track_credits['arrangement'])
        
def run_spider(spider, c):
    def f(q):
        try:
            runner = scrapy.crawler.CrawlerRunner({
                'AUTOTHROTTLE_ENABLED': True,
                'AUTOTHROTTLE_START_DELAY': 5,
                'AUTOTHROTTLE_MAX_DELAY': 30,
                'AUTOTHROTTLE_TARGET_CONCURRENCY': 60,
            #     'CONCURRENT_ITEMS': 1,
                'DEPTH_PRIORITY': -200,
                'DOWNLOAD_DELAY': 0.6,
                'HTTPCACHE_ENABLED': True,
                'HTTPCACHE_DIR': 'httpcache',
                'HTTPCACHE_IGNORE_HTTP_CODES': [429],
                'HTTPCACHE_STORAGE': 'scrapy.extensions.httpcache.FilesystemCacheStorage',
                'DOWNLOADER_MIDDLEWARES': {'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,},
                'RETRY_HTTP_CODES': [429]
            })
            deferred = runner.crawl(spider, country=c)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        raise result

In [ ]:
run_spider(DiscogsSpider, 'Serbia')

In [ ]:
run_spider(DiscogsSpider, 'Yugoslavia')

# Data Analysis

In [ ]:
cursor.execute("""SELECT genre, COUNT(*) FROM album_genre GROUP BY genre""")
for x in cursor.fetchall():
    print (x)
    
print("----------------------")
    
cursor.execute("""SELECT style, COUNT(*) FROM album_style GROUP BY style""")
for x in cursor.fetchall():
    print(x)
    
print("----------------------")

cursor.execute("""SELECT album_name, versions FROM album ORDER BY versions DESC LIMIT 20""")
for x in cursor.fetchall():
    print(x)
    
print("----------------------")

cursor.execute("""SELECT a.artist_name, AVG(b.rating) FROM artist a, album b WHERE b.rating IS NOT NULL and b.rating != 0 GROUP BY a.artist_name ORDER BY AVG(b.rating) DESC LIMIT 100""")
for x in cursor.fetchall():
    print(x)
    
print("----------------------")

cursor.execute("""SELECT a.artist_name, COUNT(v.album_id) FROM artist a, album_vocals v WHERE a.id = v.artist_id GROUP BY a.id ORDER BY COUNT(v.album_id) DESC LIMIT 100""")
for x in cursor.fetchall():
    print(x)

print("----------------------")

cursor.execute("""SELECT a.artist_name, COUNT(v.album_id) FROM artist a, album_vocals v WHERE a.id = v.artist_id GROUP BY a.id ORDER BY COUNT(v.album_id) DESC LIMIT 100""") # popravi
for x in cursor.fetchall():
    print(x)

print("----------------------")

cursor.execute("""SELECT t.track_name, COUNT(t.album_id), a.country, a.released, g.genre, a.style FROM track_list t, album a, album_genre g WHERE a.id = g.album_id GROUP BY g.album_id GROUP BY t.track_name ORDER BY COUNT(t.album_id) DESC LIMIT 100""")
for x in cursor.fetchall():
    print(x)

print("----------------------")

cursor.execute("""SELECT artist_name, sites FROM artist WHERE sites IS NOT NULL AND sites != ''""")
for x in cursor.fetchall():
    print(x)

In [ ]:
cursor.close()
conn.close()